In [4]:
import httpx
import asyncio
import time

In [35]:
symbol = "QCOM"

async def download_basics():
    
    async with httpx.AsyncClient() as client:
        
        start = time.time()
        response = await client.get(f"https://www.tipranks.com/api/stocks/getData/?name={symbol}&benchmark=1&period=3&break=1582103013435")
        json_data = response.json()
        
        return {
            "company_name": json_data["companyName"],
            "market": json_data["market"],
            "description": json_data["description"],
            "market_cap": json_data["marketCap"],
            "has_dividends": json_data["hasDividends"],
            "similar_stocks": [{"ticker": stock["ticker"], "company_name": stock["name"]} for stock in json_data["similarStocks"]],
            "yoy_change": json_data["momentum"]["momentum"] * 100,
        }

async def download_additionals():
    
    async with httpx.AsyncClient() as client:
        
        response = await client.get(f"https://market.tipranks.com/api/details/GetRealTimeQuotes/?tickers={symbol}&break=1582103013582")
        json_data = response.json()
        
        return {
            "year_low": json_data[0]["yLow"],
            "year_high": json_data[0]["yHigh"],
            "pe_ratio": json_data[0]["pe"],
            "market_cap": json_data[0]["marketCap"],
            "eps": json_data[0]["eps"],
        }
    
basic, additionals = await asyncio.gather(download_basics(), download_additionals())
basic.update(additionals)

basic

{'company_name': 'Qualcomm',
 'market': 'NASDAQ',
 'description': 'Qualcomm Inc develops digital communication technology called CDMA (Code Division Multiple Access), & owns intellectual property applicable to products that implement any version of CDMA including patents, patent applications & trade secrets.',
 'market_cap': '100.62B',
 'has_dividends': True,
 'similar_stocks': [{'ticker': 'MSFT', 'company_name': 'Microsoft'},
  {'ticker': 'SNE', 'company_name': 'Sony'},
  {'ticker': 'LEA', 'company_name': 'Lear'},
  {'ticker': 'INTC', 'company_name': 'Intel'},
  {'ticker': 'LNVGY', 'company_name': 'Lenovo Group'}],
 'yoy_change': 75.49840510366826,
 'year_low': 51.75,
 'year_high': 96.17,
 'pe_ratio': 25.5,
 'eps': 3.52}